In [4]:
# Imports
import ffmpeg
import os
import subprocess
import re
import pandas as pd
import numpy as np
from mutagen.easyid3 import EasyID3
import shlex

In [10]:
source = "TestFile.mp3"
output = "Outputs/OutFile.mp3"
check = "TestFile2.mp3"

In [18]:
spd = 1.35 # output speed
vol = 1.03 # volume rise ratio if any
bit = 16 # BitRate: 16K
frq = 24000 # sampling stream frequency

#Temporary parameters
src, out = source, output
# commands
stg1 = f'ffmpeg -i {src} -filter:a "atempo={spd},volume={vol}" -q:a 100 {out}'
stg2 = f"ffmpeg -i {src} -ac 1 -b:a {bit}k -ar {frq} -write_xing 0 {out}"

In [19]:
stg1

'ffmpeg -i TestFile.mp3 -filter:a "atempo=1.35,volume=1.03" -q:a 100 Outputs/OutFile.mp3'

In [8]:
cmnd = shlex.split(stg1)
cmnd

['ffmpeg',
 '-i',
 'TestFile.mp3',
 '-filter:a',
 'atempo=1.5,volume=1.0',
 '-q:a',
 '100',
 'OutFile.mp3']

In [9]:
# See if they are different
stg1.split(" ")

['ffmpeg',
 '-i',
 'TestFile.mp3',
 '-filter:a',
 '"atempo=1.5,volume=1.0"',
 '-q:a',
 '100',
 'OutFile.mp3']

**Trying to execute `ffmpeg` commands from python**

Here: https://stackoverflow.com/a/67998291/5820024

In [20]:
#Stage 1
src, out = source, output
stg1 = f'ffmpeg -i {src} -filter:a "atempo={spd},volume={vol}" -q:a 100 {out}'
subprocess.call(stg1, shell=True)

0

In [23]:
# Stage 2
src, out = out, 'Outputs/OutFileStage2.mp3'
stg2 = f"ffmpeg -i {src} -ac 1 -b:a {bit}k -ar {frq} -write_xing 0 {out}"

subprocess.call(stg2, shell=True)

0

In [24]:
stg2

'ffmpeg -i Outputs/OutFile.mp3 -ac 1 -b:a 16k -ar 24000 -write_xing 0 Outputs/OutFileStage2.mp3'

In [27]:
"ffmpeg -i Fast\023_Al_Mu_Z_minun\23AlMu_Z_minun13o14T7.9o9.3.mp3 -ac 1 -b:a 16k -ar 24000 -write_xing 0 Final\023_Al_Mu_Z_minun\23Nesr13o14T7.9o9.3.mp3"
"ffmpeg -i Outputs/OutFile.mp3 -ac 1 -b:a 16k -ar 24000 -write_xing 0 Outputs/OutFileStage2.mp3" ==stg2

True

Trying to find the best ffmpeg commands,

from these posts:
1. https://stackoverflow.com/a/12952172/5820024
2. https://stackoverflow.com/a/39789131/5820024
3. https://superuser.com/a/553049/724450
4. https://superuser.com/a/704535/724450
    

In [32]:
src = source
out = 'Outputs/OutFileOneShot.mp3'
vol = 0.95
ffmpeg_command = f'ffmpeg -y -i {src} -filter:a "atempo={spd},volume={vol}" '
ffmpeg_command += f'-vn -ar 44100 -ac 2 -b:a 16k -write_xing 0 {out}'
print(ffmpeg_command)
performed = subprocess.call(ffmpeg_command, shell=True)
if performed==0:
    print("*Done*")
else:
    print("failed :-\(")

ffmpeg -y -i TestFile.mp3 -filter:a "atempo=1.35,volume=0.95" -vn -ar 44100 -ac 2 -b:a 16k -write_xing 0 Outputs/OutFileOneShot.mp3
*Done*


**Applying the successful command to all files**

Benefitted from:https://stackoverflow.com/a/30255302/5820024

In [34]:
folder = "X:/Music/RightlyGuidedCalifate/output/"
print(os.listdir(folder))

['Group_01', 'Group_02', 'Group_03', 'Group_04', 'Group_05', 'Group_06', 'Group_07', 'Group_08', 'Group_09', 'Group_10', 'Group_11', 'Group_12', 'Group_13', 'Group_14', 'Group_15', 'Group_16']


In [49]:
# add the star to decompress the list
os.path.join(*['X:', 'Music', 'RightlyGuidedCalifate', 'output', 'Group_01'])

'X:Music\\RightlyGuidedCalifate\\output\\Group_01'

In [61]:
i = 0
vol = 0.95
failed_list = []
for subdir, dirs, files in os.walk(folder):
    for file in files:
        i += 1
        if i > 0:  # It was <3, to limit outputs for testing, now to execute all!
            #print(subdir, os.path.join(subdir, file))
            subdir_list = subdir.split('/')
            output_list = [element for element in subdir_list]  # Deep copy
            output_list[3] = 'output2'
            output_folder = os.path.join(*output_list)
            exists = os.path.exists(output_folder)
            if not exists:
                os.makedirs(output_folder)
            # print(file, output_list)#,subdir_list )#subdir.split('/'))
            src = os.path.join(subdir, file)
            out = os.path.join(output_folder, file)
            # print(src, out)

            ffmpeg_command = f'ffmpeg -y -i {src} -filter:a "atempo={spd},volume={vol}" '
            ffmpeg_command += f'-vn -ar 44100 -ac 2 -b:a 16k -write_xing 0 {out}'
            print(i, file, end=" >>> ")
            performed = subprocess.call(ffmpeg_command, shell=True)
            if performed == 0:
                print("*Done*", end=" ~")
            else:
                failed_list.append((subdir, file))
                print("failed :-\(")

1 RGC001_0.mp3 >>> *Done* ~2 RGC001_1.mp3 >>> *Done* ~3 RGC001_2.mp3 >>> *Done* ~4 RGC001_3.mp3 >>> *Done* ~5 RGC001_4.mp3 >>> *Done* ~6 RGC001_5.mp3 >>> *Done* ~7 RGC001_6.mp3 >>> *Done* ~8 RGC001_7.mp3 >>> *Done* ~9 RGC001_8.mp3 >>> *Done* ~10 RGC001_9.mp3 >>> *Done* ~11 RGC002_0.mp3 >>> *Done* ~12 RGC002_1.mp3 >>> *Done* ~13 RGC002_2.mp3 >>> *Done* ~14 RGC002_3.mp3 >>> *Done* ~15 RGC002_4.mp3 >>> *Done* ~16 RGC002_5.mp3 >>> *Done* ~17 RGC002_6.mp3 >>> *Done* ~18 RGC002_7.mp3 >>> *Done* ~19 RGC002_8.mp3 >>> *Done* ~20 RGC003_0.mp3 >>> *Done* ~21 RGC003_1.mp3 >>> *Done* ~22 RGC003_2.mp3 >>> *Done* ~23 RGC003_3.mp3 >>> *Done* ~24 RGC003_4.mp3 >>> *Done* ~25 RGC003_5.mp3 >>> *Done* ~26 RGC003_6.mp3 >>> *Done* ~27 RGC003_7.mp3 >>> *Done* ~28 RGC003_8.mp3 >>> *Done* ~29 RGC003_9.mp3 >>> *Done* ~30 RGC004_0.mp3 >>> *Done* ~31 RGC004_1.mp3 >>> *Done* ~32 RGC004_2.mp3 >>> *Done* ~33 RGC004_3.mp3 >>> *Done* ~34 RGC004_4.mp3 >>> *Done* ~35 RGC004_5.mp3 >>> *Done* ~36 RGC004_6.mp3 >>> *Done* ~3

*Done* ~288 RGC029_8.mp3 >>> *Done* ~289 RGC029_9.mp3 >>> *Done* ~290 RGC030_0.mp3 >>> *Done* ~291 RGC030_1.mp3 >>> *Done* ~292 RGC030_2.mp3 >>> *Done* ~293 RGC030_3.mp3 >>> *Done* ~294 RGC030_4.mp3 >>> *Done* ~295 RGC030_5.mp3 >>> *Done* ~296 RGC030_6.mp3 >>> *Done* ~297 RGC030_7.mp3 >>> *Done* ~298 RGC030_8.mp3 >>> *Done* ~299 RGC030_9.mp3 >>> *Done* ~300 RGC031_0.mp3 >>> *Done* ~301 RGC031_1.mp3 >>> *Done* ~302 RGC031_2.mp3 >>> *Done* ~303 RGC031_3.mp3 >>> *Done* ~304 RGC031_4.mp3 >>> *Done* ~305 RGC031_5.mp3 >>> *Done* ~306 RGC031_6.mp3 >>> *Done* ~307 RGC031_7.mp3 >>> *Done* ~308 RGC031_8.mp3 >>> *Done* ~309 RGC031_9.mp3 >>> *Done* ~310 RGC032_0.mp3 >>> *Done* ~311 RGC032_1.mp3 >>> *Done* ~312 RGC032_2.mp3 >>> *Done* ~313 RGC032_3.mp3 >>> *Done* ~314 RGC032_4.mp3 >>> *Done* ~315 RGC032_5.mp3 >>> *Done* ~316 RGC032_6.mp3 >>> *Done* ~317 RGC032_7.mp3 >>> *Done* ~318 RGC032_8.mp3 >>> *Done* ~319 RGC032_9.mp3 >>> *Done* ~320 RGC033_0.mp3 >>> *Done* ~321 RGC033_1.mp3 >>> *Done* ~322 RG

*Done* ~571 RGC058_1.mp3 >>> *Done* ~572 RGC058_2.mp3 >>> *Done* ~573 RGC058_3.mp3 >>> *Done* ~574 RGC058_4.mp3 >>> *Done* ~575 RGC058_5.mp3 >>> *Done* ~576 RGC058_6.mp3 >>> *Done* ~577 RGC058_7.mp3 >>> *Done* ~578 RGC058_8.mp3 >>> *Done* ~579 RGC058_9.mp3 >>> *Done* ~580 RGC059_0.mp3 >>> *Done* ~581 RGC059_1.mp3 >>> *Done* ~582 RGC059_2.mp3 >>> *Done* ~583 RGC059_3.mp3 >>> *Done* ~584 RGC059_4.mp3 >>> *Done* ~585 RGC059_5.mp3 >>> *Done* ~586 RGC059_6.mp3 >>> *Done* ~587 RGC059_7.mp3 >>> *Done* ~588 RGC059_8.mp3 >>> *Done* ~589 RGC059_9.mp3 >>> *Done* ~590 RGC060_0.mp3 >>> *Done* ~591 RGC060_1.mp3 >>> *Done* ~592 RGC060_2.mp3 >>> *Done* ~593 RGC060_3.mp3 >>> *Done* ~594 RGC060_4.mp3 >>> *Done* ~595 RGC060_5.mp3 >>> *Done* ~596 RGC060_6.mp3 >>> *Done* ~597 RGC060_7.mp3 >>> *Done* ~598 RGC060_8.mp3 >>> *Done* ~599 RGC060_9.mp3 >>> *Done* ~600 RGC061_0.mp3 >>> *Done* ~601 RGC061_1.mp3 >>> *Done* ~602 RGC061_2.mp3 >>> *Done* ~603 RGC061_3.mp3 >>> *Done* ~604 RGC061_4.mp3 >>> *Done* ~605 RG

*Done* ~854 RGC086_2.mp3 >>> *Done* ~855 RGC086_3.mp3 >>> *Done* ~856 RGC086_4.mp3 >>> *Done* ~857 RGC086_5.mp3 >>> *Done* ~858 RGC086_6.mp3 >>> *Done* ~859 RGC086_7.mp3 >>> *Done* ~860 RGC086_8.mp3 >>> *Done* ~861 RGC086_9.mp3 >>> *Done* ~862 RGC087_0.mp3 >>> *Done* ~863 RGC087_1.mp3 >>> *Done* ~864 RGC087_2.mp3 >>> *Done* ~865 RGC087_3.mp3 >>> *Done* ~866 RGC087_4.mp3 >>> *Done* ~867 RGC087_5.mp3 >>> *Done* ~868 RGC087_6.mp3 >>> *Done* ~869 RGC087_7.mp3 >>> *Done* ~870 RGC087_8.mp3 >>> *Done* ~871 RGC087_9.mp3 >>> *Done* ~872 RGC088_0.mp3 >>> *Done* ~873 RGC088_1.mp3 >>> *Done* ~874 RGC088_2.mp3 >>> *Done* ~875 RGC088_3.mp3 >>> *Done* ~876 RGC088_4.mp3 >>> *Done* ~877 RGC088_5.mp3 >>> *Done* ~878 RGC088_6.mp3 >>> *Done* ~879 RGC088_7.mp3 >>> *Done* ~880 RGC088_8.mp3 >>> *Done* ~881 RGC088_9.mp3 >>> *Done* ~882 RGC089_0.mp3 >>> *Done* ~883 RGC089_1.mp3 >>> *Done* ~884 RGC089_2.mp3 >>> *Done* ~885 RGC089_3.mp3 >>> *Done* ~886 RGC089_4.mp3 >>> *Done* ~887 RGC089_5.mp3 >>> *Done* ~888 RG